# Util-kit

In [1]:
import io
import pandas as pd
import numpy as np
import itertools
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt     
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
!pip install pandas numpy
!pip install -U scikit-learn
!pip install -U matplotlib

In [3]:
attacks = [
    {"start":"2019-07-20 15:08:46.004013+08:00", "end":"2019-07-20 15:10:31.004013+08:00", "cols": ["FIT 401", "UV401"]},
    {"start":"2019-07-20 15:15:00.004013+08:00", "end":"2019-07-20 15:19:32.004013+08:00", "cols": ["LIT 301"]},
    {"start":"2019-07-20 15:26:57.004013+08:00", "end":"2019-07-20 15:30:48.004013+08:00", "cols": ["P601 Status"]},
    {"start":"2019-07-20 15:38:50.004013+08:00", "end":"2019-07-20 15:46:20.004013+08:00", "cols": ["MV201", "P101 Status"]},
    {"start":"2019-07-20 15:54:00.004013+08:00", "end":"2019-07-20 15:56:00.004013+08:00", "cols": ["MV 501"]},
    {"start":"2019-07-20 16:02:56.004013+08:00", "end":"2019-07-20 16:16:18.004013+08:00", "cols": ["P301 Status"]},
]

In [4]:
global raw_df
raw_df = None
def read_data():
    global raw_df
    if raw_df is None:
      raw_df = pd.read_excel('SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0'], index_col = 'GMT +0')
      raw_df = raw_df.rename(columns=lambda x: x.strip())
      raw_df.index = raw_df.index.tz_convert('Asia/Singapore') + pd.Timedelta(minutes=2)
    return raw_df.copy()

In [5]:
def keep_columns(all, to_keep=["FIT 401", "LIT 301"]):
    if len(to_keep):
      return list(set(all).intersection(set(to_keep)))
    return all

In [6]:
def split_feature(df):
    cat_features = []
    numeric_features = []
    for col in df.columns:
        if len(df[col].unique()) < 4:
          cat_features.append(col)
        else: numeric_features.append(col)
    return cat_features, numeric_features

In [7]:
def preprocess_cat(df): 
    df = df.copy() 
    df['LS 201'] = np.where(df['LS 201'] == 'Active', 1, 0)
    df['LS 202'] = np.where(df['LS 202'] == 'Active', 1, 0)
    df['LSL 203'] = np.where(df['LSL 203'] == 'Inactive', 0, 1)
    df['LSLL 203'] = np.where(df['LSLL 203'] == 'Active', 1, 0)
    df['LS 401'] = np.where(df['LS 401'] == 'Active', 1, 0)
    df['LSH 601'] = np.where(df['LSH 601'] == 'Active', 1, 0)
    df['LSH 602'] = np.where(df['LSH 602'] == 'Active', 1, 0)
    df['LSH 603'] = np.where(df['LSH 603'] == 'Active', 1, 0)
    df['LSL 601'] = np.where(df['LSL 601'] == 'Active', 1, 0)
    df['LSL 602'] = np.where(df['LSL 602'] == 'Active', 1, 0)
    df['LSL 603'] = np.where(df['LSL 603'] == 'Active', 1, 0)
    return df

In [8]:
def preprocess(df, cols, to_plot=False):
    
    enc_df = None
    minmax_scale = None
    cat_features, numeric_features = split_feature(df)
    cat_features = keep_columns(cat_features, cols)
    numeric_features = keep_columns(numeric_features, cols)
    cols = numeric_features + cat_features

    df = df.copy()
    df = df[cols]
    #OneHotEncode categorical data
    if len(cat_features) > 0:
      encoder = OneHotEncoder(sparse = False)
      enc_df = pd.DataFrame(encoder.fit_transform(df[cat_features]))
      enc_df.index = df.index
    
    if len(numeric_features) > 0:
        # Normalize numerical data
        minmax_scale = pd.DataFrame(MinMaxScaler().fit_transform(df[numeric_features]))
        minmax_scale.index = df.index 
    
    if isinstance(enc_df, pd.DataFrame) and  isinstance(minmax_scale, pd.DataFrame):
        set_val = pd.concat([enc_df, minmax_scale], axis=1, ignore_index=True) 
        set_val = set_val.reindex(df.index)
    elif isinstance(enc_df, pd.DataFrame):
         set_val = enc_df
    elif isinstance(minmax_scale, pd.DataFrame):
         set_val = minmax_scale
    else:
         set_val = -1

    return set_val

In [9]:
def get_clean_df(cols=[], to_plot=False):
    df = read_data()
    df = preprocess_cat(df)
    sel = VarianceThreshold(threshold=0)
    sel.fit(df)  # fit finds the features with zero variance
    df = df[df.columns[sel.get_support()]]
    # print("Number of columns " + df.columns)
    df = preprocess(df, cols)
    return df

In [10]:
def list_to_str(cols):
    return "_".join(cols).replace(" ", "_")

In [11]:
def evaluate(df, start, end, ev_col):
    x = df[((df.index<=start)&(df.index<=end))]
    return len(x[x[ev_col]==-1])/len(x), len(df[df[ev_col]==-1])

In [72]:
def evaluate_all_features(df, attacks, ev_col):
    result = []
    ratio = 0
    for attack in attacks:
      x = df[((df.index<=attack["start"])&(df.index<=attack["end"]))]
      ratio += len(x[x[ev_col]==-1])/len(x)
    return ratio/len(attacks)

In [73]:
def plot_ts(x, y, colors, title):
    fig = plt.figure()
    fig.set_figwidth(25)
    ax1 = fig.add_subplot(111)
    ax1.set_title(title)
    sc = ax1.scatter(x,y, s=10,c=colors)
    plt.colorbar(sc)
    for attack in attacks:
      ax1.fill_between(x, 0, 1,where=(x >= attack["start"] )& (x<=attack["end"]),
                     color='red', alpha=0.5, transform=ax1.get_xaxis_transform())
    plt.show()

In [74]:
def unsupervised_iso_grid_search(df, max_samples, contaminations, n_estimators, max_features, attack):
    hypyer_parameters = [max_samples, contaminations, n_estimators, max_features]
    modified_df = df.copy()
    df_orginal = df.copy()
    best_model = {"model_name": None, "anomaly_percentage":0, "anomlaies_num":0}
    for a, b, c, d in itertools.product(*hypyer_parameters):
        model_name = f"iso_{a}_{b}_{c}_{d}"
        model = IsolationForest(max_samples=a, contamination=b, n_estimators=c, max_features=d)
        model.fit(df_orginal)
        modified_df[model_name] = model.predict(df_orginal)
        anomaly_percentage, anomlaies_num = evaluate(modified_df, attack["start"], attack["end"], model_name)
        if best_model["anomaly_percentage"] <  anomaly_percentage:
           best_model["anomaly_percentage"] = anomaly_percentage
           best_model["model_name"] = model_name
           best_model["anomlaies_num"] = anomlaies_num
        # print(f"{model_name}: {anomaly_percentage} - {best_model['anomlaies_num']}")  
    return best_model      

In [75]:
def fit_predict(all_features=False):
  if all_features is False:
      for x in attacks:
          df = get_clean_df(x["cols"])
          features = list_to_str(x["cols"])
          best_model = unsupervised_iso_grid_search(df, max_samples=[100, 200, 500, 'auto'],
                                    n_estimators=[50, 100, 200],
                                    contaminations=[0.1, 0.2, 0.3, 0.5,'auto'],
                                    max_features=[1],
                                    attack=x) # attack info used for evaluation
          print(f"Best model on {features} is {best_model['model_name']} with ratio {best_model['anomaly_percentage']} and {best_model['anomlaies_num']} total anomalies")
  else:
      df = get_clean_df(['FIT 401',
            'UV401',
            'LIT 301',
            'P601 Status',
            'MV201',
            'P101 Status',
            'MV 501',
            'P301 Status'])
      best_model = unsupervised_iso_grid_search_all_features(df, max_samples=['auto'],
                                    n_estimators=[50],
                                    max_features=[1],
                                    contaminations=['auto']) # attack info used for evaluation
      print(best_model)

In [76]:
def fit_predict_single(df, max_samples=100, contaminations='auto', n_estimators=50, max_features=1, features=[]):
    df_orginal = df.copy()
    model_name = f"iso_{max_samples}_{contaminations}_{n_estimators}_{max_features}"
    model = IsolationForest(max_samples=max_samples, contamination=contaminations, n_estimators=n_estimators, max_features=max_features)
    model.fit(df_orginal)
    df[model_name] = model.predict(df_orginal)
    res = evaluate_all_features(df, attacks, model_name)
#     plot_df = read_data()
#     plot_df = preprocess_cat(plot_df)
#     plot_df = plot_df[features]
#     plot_df[features] = MinMaxScaler().fit_transform(plot_df[features])
#     plot_df.index = df.index

#     for feature in features:
#       plot_ts(plot_df.index, plot_df[feature], df[model_name], title=feature)
    return res, model, model_name

In [80]:
def unsupervised_iso_grid_search_all_features(df, max_samples=[100, 200, 500, 'auto'],
                                    n_estimators=[50, 100, 200],
                                    contaminations=[0.1, 0.2, 0.3, 0.5,'auto'],
                                    max_features=[1],attacks=attacks):
    hypyer_parameters = [max_samples, contaminations, n_estimators, max_features]
    modified_df = df.copy()
    df_orginal = df.copy()
    best_model = {"model":None, "name":None, "res":0}
    for a, b, c, d in itertools.product(*hypyer_parameters):
        print(a,b,c,d)
        res, model, model_name = fit_predict_single(df, max_samples=a, contaminations=b, n_estimators=50, max_features=1, features=features)
        print(f"{model_name}: {res}")
        if res > best_model["res"]:
            best_model["res"] = res
            best_model["name"] = model_name
            best_model["model"] = model
    print(f"best model is {best_model['name']}: {best_model['res']}")
    return best_model

In [81]:
import warnings
warnings.filterwarnings('ignore')

In [82]:
features = ['FIT 401',
            'UV401',
            'LIT 301',
            'P601 Status',
            'MV201',
            'P101 Status',
            'MV 501',
            'P301 Status']
df = get_clean_df([])
unsupervised_iso_grid_search_all_features(df)

100 0.1 50 1
iso_100_0.1_50_1: 0.1071052457245044
100 0.1 100 1
iso_100_0.1_50_1: 0.10217766722071653
100 0.1 200 1
iso_100_0.1_50_1: 0.09358758028972881
100 0.2 50 1
iso_100_0.2_50_1: 0.1510752587926951
100 0.2 100 1
iso_100_0.2_50_1: 0.15578924345687448
100 0.2 200 1
iso_100_0.2_50_1: 0.16111234817295625
100 0.3 50 1
iso_100_0.3_50_1: 0.28781228833985784
100 0.3 100 1
iso_100_0.3_50_1: 0.21081249317129247
100 0.3 200 1
iso_100_0.3_50_1: 0.24848429956927073
100 0.5 50 1
iso_100_0.5_50_1: 0.3732991911159879
100 0.5 100 1
iso_100_0.5_50_1: 0.39812627008811585
100 0.5 200 1
iso_100_0.5_50_1: 0.436273923506376
100 auto 50 1
iso_100_auto_50_1: 0.13901781587643455
100 auto 100 1
iso_100_auto_50_1: 0.1863534393078794
100 auto 200 1
iso_100_auto_50_1: 0.20924715648001277
200 0.1 50 1
iso_200_0.1_50_1: 0.05718786501743631
200 0.1 100 1
iso_200_0.1_50_1: 0.1051844624872761
200 0.1 200 1
iso_200_0.1_50_1: 0.0876158037057293
200 0.2 50 1
iso_200_0.2_50_1: 0.14737472175015823
200 0.2 100 1
iso_200

{'model': IsolationForest(contamination=0.5, max_features=1, max_samples=500,
                 n_estimators=50),
 'name': 'iso_500_0.5_50_1',
 'res': 0.4901814362021631}